## XỬ LÍ - LÀM SẠCH DỮ LIỆU


#### I. Mô tả dữ liệu 

In [46]:
import pandas as pd
import numpy as np

In [47]:
# Đọc dữ liệu từ file csv
stroke_preditition_df = pd.read_csv('healthcare-dataset-stroke-data.csv')

In [48]:
# Kiểm tra thông tin sơ bộ của Data Frame
stroke_preditition_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


Dữ liệu được lấy từ bộ dữ liệu chăm sóc sức khỏe (Healthcare Dataset - Stroke Data), gồm nhiều thông tin liên quan đến hồ sơ y tế của bệnh nhân.
Một số cột quan trọng trong tập dữ liệu gồm:

- id: Mã định danh bệnh nhân (không dùng trong dự đoán).
- gender: Giới tính.
- age: Tuổi.
- hypertension: Tình trạng huyết áp cao.
- heart_disease: Bệnh tim.
- ever_married: Tình trạng hôn nhân.
- work_type: Loại công việc.
- Residence_type: Khu vực sống (nông thôn/thành thị).
- avg_glucose_level: Mức đường huyết trung bình.
- bmi: Chỉ số khối cơ thể.
- smoking_status: Tình trạng hút thuốc.
stroke: Cột mục tiêu (1 nếu có đột quỵ, 0 nếu không).

#### II. Thực hiện tiền xử lí

1. Xử lý Dữ liệu thiếu (Handling Missing Values):

In [49]:

# Xử lý giá trị thiếu ở cột BMI
stroke_preditition_df['bmi'].fillna(stroke_preditition_df['bmi'].median(), inplace=True)



C:\Users\hary0\AppData\Local\Temp\ipykernel_3220\2996480653.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  stroke_preditition_df['bmi'].fillna(stroke_preditition_df['bmi'].median(), inplace=True)


- Vì BMI là dữ liệu số, thường phân bố lệch nhẹ, nên median (trung vị) sẽ an toàn hơn mean (trung bình) khi có outlier.

2. Mã hóa dữ liệu (Encoding Categorical Data):

In [50]:
# Các cột cần mã hóa
categorical_cols = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

# Tạo DataFrame chứa các cột đã mã hóa
encoded = pd.get_dummies(stroke_preditition_df[categorical_cols], drop_first=True)

# Ghép lại vào stroke_preditition_df gốc, giữ nguyên thứ tự các cột ban đầu
stroke_preditition_df = pd.concat([stroke_preditition_df.drop(columns=categorical_cols), encoded], axis=1)


- Áp dụng phương pháp One-Hot Encoding để chuyển các giá trị phân loại thành các biến nhị phân (0 và 1). Mỗi giá trị trong một cột được mã hóa thành một cột mới đại diện riêng biệt.
Việc này giúp mô hình hiểu đúng bản chất của dữ liệu, tránh việc hiểu sai thứ tự giữa các giá trị (như khi dùng Label Encoding), đồng thời cải thiện khả năng học và dự đoán của mô hình.

3. Xử lý dữ liệu không liên quan (Handling Irrelevant Features):

In [51]:
# Xóa cột id không liên quan
stroke_preditition_df = stroke_preditition_df.drop( columns= 'id', inplace = False)

- Trong bộ dữ liệu, cột id chỉ đóng vai trò là mã số định danh cho từng bản ghi (record). Nó không mang thông tin thực chất nào về đặc điểm của bệnh nhân hoặc kết quả bệnh lý, nên không có giá trị dự đoán trong mô hình học máy.

4. Xử lý dữ liệu ngoại lai (Outlier)

In [52]:
numeric_cols = ['age', 'avg_glucose_level', 'bmi']

for col in numeric_cols:
    Q1 = stroke_preditition_df[col].quantile(0.25)
    Q3 = stroke_preditition_df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Dùng np.clip để giới hạn giá trị trong khoảng [lower_bound, upper_bound]
    stroke_preditition_df[col] = np.clip(stroke_preditition_df[col], lower_bound, upper_bound)

- Xử lý giá trị ngoại lai (outlier) giúp mô hình học máy hoạt động ổn định và chính xác hơn. Vì các giá trị quá lớn hoặc quá nhỏ có thể làm sai lệch trung bình, phương sai và hướng học của mô hình, dẫn đến kết quả dự đoán kém tin cậy.

#### III. Cân bằng dữ liệu